In [ ]:
! pip install pyspark

In [ ]:
from google.colab import files

uploaded = files.upload()  # Une boîte de dialogue apparaîtra pour choisir le fichier


Saving ngram.csv to ngram.csv


In [ ]:
import pandas as pd

df = pd.read_csv("ngram.csv")

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("NGramAnalysis").getOrCreate()


In [ ]:
df = spark.read.csv("ngram.csv", sep="\t", header=False, inferSchema=True)
df.show(5)



+--------+----+---+---+---+
|     _c0| _c1|_c2|_c3|_c4|
+--------+----+---+---+---+
|! $17.95|1985|  1|  1|  1|
|! $17.95|1987|  1|  1|  1|
|! $17.95|1990|  1|  1|  1|
|! $17.95|1991|  1|  1|  1|
|! $17.95|1992|  5|  5|  5|
+--------+----+---+---+---+
only showing top 5 rows



In [ ]:
df = df.toDF("Ngram", "Year", "Count", "Pages", "Books")

In [ ]:
df.printSchema()
df.show(5)


root
 |-- Ngram: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Count: integer (nullable = true)
 |-- Pages: integer (nullable = true)
 |-- Books: integer (nullable = true)

+--------+----+-----+-----+-----+
|   Ngram|Year|Count|Pages|Books|
+--------+----+-----+-----+-----+
|! $17.95|1985|    1|    1|    1|
|! $17.95|1987|    1|    1|    1|
|! $17.95|1990|    1|    1|    1|
|! $17.95|1991|    1|    1|    1|
|! $17.95|1992|    5|    5|    5|
+--------+----+-----+-----+-----+
only showing top 5 rows



In [ ]:
df.createOrReplaceTempView("ngram_table")
result_sql = spark.sql("SELECT * FROM ngram_table WHERE Count > 5")
result_sql.show()

result_df = df.filter(df["Count"] > 5)
result_df.show()


+--------+----+-----+-----+-----+
|   Ngram|Year|Count|Pages|Books|
+--------+----+-----+-----+-----+
|! $17.95|1997|    6|    5|    5|
|! $17.95|1999|   11|   10|   10|
|! $17.95|2000|   11|    9|    9|
|! $17.95|2004|   14|   14|   14|
|! $17.95|2005|   13|   13|   13|
|    ! 09|1899|    6|    6|    5|
|    ! 09|1916|    7|    7|    4|
|    ! 09|1936|    6|    6|    6|
|    ! 09|1997|    6|    5|    5|
|    ! 09|1999|   11|   10|   10|
|    ! 09|2000|   11|    9|    9|
|    ! 09|2004|   14|   14|   14|
|    ! 09|2005|   13|   13|   13|
+--------+----+-----+-----+-----+

+--------+----+-----+-----+-----+
|   Ngram|Year|Count|Pages|Books|
+--------+----+-----+-----+-----+
|! $17.95|1997|    6|    5|    5|
|! $17.95|1999|   11|   10|   10|
|! $17.95|2000|   11|    9|    9|
|! $17.95|2004|   14|   14|   14|
|! $17.95|2005|   13|   13|   13|
|    ! 09|1899|    6|    6|    5|
|    ! 09|1916|    7|    7|    4|
|    ! 09|1936|    6|    6|    6|
|    ! 09|1997|    6|    5|    5|
|    ! 09|199

In [ ]:
result_sql = spark.sql("""
    SELECT Year, COUNT(Ngram) AS total_Ngrams
    FROM ngram_table
    GROUP BY Year
    ORDER BY Year
""")
result_sql.show()

result_df = df.groupBy("Year").count().withColumnRenamed("count", "total_Ngrams")
result_df.orderBy("Year").show()



+----+------------+
|Year|total_Ngrams|
+----+------------+
|1780|           1|
|1803|           1|
|1806|           1|
|1823|           1|
|1824|           1|
|1825|           1|
|1829|           1|
|1830|           1|
|1831|           1|
|1833|           1|
|1834|           1|
|1836|           1|
|1839|           1|
|1840|           1|
|1841|           1|
|1845|           1|
|1847|           1|
|1848|           1|
|1856|           1|
|1857|           1|
+----+------------+
only showing top 20 rows

+----+------------+
|Year|total_Ngrams|
+----+------------+
|1780|           1|
|1803|           1|
|1806|           1|
|1823|           1|
|1824|           1|
|1825|           1|
|1829|           1|
|1830|           1|
|1831|           1|
|1833|           1|
|1834|           1|
|1836|           1|
|1839|           1|
|1840|           1|
|1841|           1|
|1845|           1|
|1847|           1|
|1848|           1|
|1856|           1|
|1857|           1|
+----+------------+
only showing t

In [ ]:
result_sql = spark.sql("""
    SELECT Year, Ngram, Count
    FROM ngram_table
    WHERE Count = (SELECT MAX(Count) FROM ngram_table AS sub WHERE sub.Year = ngram_table.Year)
    ORDER BY Year
""")
result_sql.show()

from pyspark.sql.window import Window
from pyspark.sql.functions import col, max, row_number

window_spec = Window.partitionBy("Year").orderBy(col("Count").desc())

df_ranked = df.withColumn("rank", row_number().over(window_spec))

result_df = df_ranked.filter(col("rank") == 1).select("Year", "Ngram", "Count").orderBy("Year")
result_df.show()




+----+-----+-----+
|Year|Ngram|Count|
+----+-----+-----+
|1780| ! 09|    1|
|1803| ! 09|    1|
|1806| ! 09|    1|
|1823| ! 09|    1|
|1824| ! 09|    1|
|1825| ! 09|    1|
|1829| ! 09|    3|
|1830| ! 09|    2|
|1831| ! 09|    1|
|1833| ! 09|    1|
|1834| ! 09|    4|
|1836| ! 09|    1|
|1839| ! 09|    1|
|1840| ! 09|    1|
|1841| ! 09|    2|
|1845| ! 09|    1|
|1847| ! 09|    2|
|1848| ! 09|    1|
|1856| ! 09|    1|
|1857| ! 09|    2|
+----+-----+-----+
only showing top 20 rows

+----+-----+-----+
|Year|Ngram|Count|
+----+-----+-----+
|1780| ! 09|    1|
|1803| ! 09|    1|
|1806| ! 09|    1|
|1823| ! 09|    1|
|1824| ! 09|    1|
|1825| ! 09|    1|
|1829| ! 09|    3|
|1830| ! 09|    2|
|1831| ! 09|    1|
|1833| ! 09|    1|
|1834| ! 09|    4|
|1836| ! 09|    1|
|1839| ! 09|    1|
|1840| ! 09|    1|
|1841| ! 09|    2|
|1845| ! 09|    1|
|1847| ! 09|    2|
|1848| ! 09|    1|
|1856| ! 09|    1|
|1857| ! 09|    2|
+----+-----+-----+
only showing top 20 rows



In [ ]:
result_sql = spark.sql("""
    SELECT Ngram
    FROM ngram_table
    GROUP BY Ngram
    HAVING COUNT(DISTINCT Year) = 20
""")
result_sql.show()

from pyspark.sql.functions import countDistinct

result_df = df.groupBy("Ngram") \
              .agg(countDistinct("Year").alias("year_count")) \
              .filter("year_count = 20") \
              .select("Ngram")

result_df.show()



+--------+
|   Ngram|
+--------+
|! $17.95|
+--------+

+--------+
|   Ngram|
+--------+
|! $17.95|
+--------+



In [ ]:
result_sql = spark.sql("""
    SELECT Ngram
    FROM ngram_table
    WHERE SPLIT(Ngram, ' ')[0] LIKE '%!%'
      AND SPLIT(Ngram, ' ')[1] LIKE '%9%'
""")
result_sql.show()


from pyspark.sql.functions import split, col

df_split = df.withColumn("first_part", split(col("Ngram"), " ")[0]) \
             .withColumn("second_part", split(col("Ngram"), " ")[1])

result_df = df_split.filter(
    col("first_part").contains("!") & col("second_part").contains("9")
).select("Ngram")

result_df.show()



+--------+
|   Ngram|
+--------+
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
+--------+
only showing top 20 rows

+--------+
|   Ngram|
+--------+
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
|! $17.95|
+--------+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import countDistinct

result_sql = spark.sql("""
    WITH total_years AS (
        SELECT COUNT(DISTINCT Year) AS total FROM ngram_table
    )
    SELECT Ngram
    FROM ngram_table
    GROUP BY Ngram
    HAVING COUNT(DISTINCT Year) = (SELECT total FROM total_years)
""")
result_sql.show()



total_years = df.select(countDistinct("Year")).collect()[0][0]

result_df = df.groupBy("Ngram") \
              .agg(countDistinct("Year").alias("year_count")) \
              .filter(col("year_count") == total_years) \
              .select("Ngram")

result_df.show()



+-----+
|Ngram|
+-----+
| ! 09|
+-----+

+-----+
|Ngram|
+-----+
| ! 09|
+-----+



In [ ]:
result_sql = spark.sql("""
    SELECT Ngram, Year, SUM(Pages) AS total_pages, SUM(Books) AS total_books
    FROM ngram_table
    GROUP BY Ngram, Year
    ORDER BY Ngram ASC
""")
result_sql.show()


+--------+----+-----------+-----------+
|   Ngram|Year|total_pages|total_books|
+--------+----+-----------+-----------+
|! $17.95|2002|          5|          5|
|! $17.95|2007|          2|          2|
|! $17.95|1985|          1|          1|
|! $17.95|1990|          1|          1|
|! $17.95|1993|          2|          2|
|! $17.95|2003|          2|          2|
|! $17.95|2000|          9|          9|
|! $17.95|2005|         13|         13|
|! $17.95|1999|         10|         10|
|! $17.95|1992|          5|          5|
|! $17.95|1996|          2|          2|
|! $17.95|1997|          5|          5|
|! $17.95|1995|          1|          1|
|! $17.95|1987|          1|          1|
|! $17.95|2004|         14|         14|
|! $17.95|2006|          5|          5|
|! $17.95|1998|          3|          3|
|! $17.95|1991|          1|          1|
|! $17.95|2008|          2|          2|
|! $17.95|2001|          4|          4|
+--------+----+-----------+-----------+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import countDistinct

result_sql = spark.sql("""
    SELECT Year, COUNT(DISTINCT Ngram) AS total_distinct_Ngrams
    FROM ngram_table
    GROUP BY Year
    ORDER BY Year DESC
""")
result_sql.show()


result_df = df.groupBy("Year") \
              .agg(countDistinct("Ngram").alias("total_distinct_Ngrams")) \
              .orderBy("Year", ascending=False)
result_df.show()



+----+---------------------+
|Year|total_distinct_Ngrams|
+----+---------------------+
|2008|                    2|
|2007|                    2|
|2006|                    2|
|2005|                    2|
|2004|                    2|
|2003|                    2|
|2002|                    2|
|2001|                    2|
|2000|                    2|
|1999|                    2|
|1998|                    2|
|1997|                    2|
|1996|                    2|
|1995|                    2|
|1993|                    2|
|1992|                    2|
|1991|                    2|
|1990|                    2|
|1987|                    2|
|1985|                    2|
+----+---------------------+
only showing top 20 rows

+----+---------------------+
|Year|total_distinct_Ngrams|
+----+---------------------+
|2008|                    2|
|2007|                    2|
|2006|                    2|
|2005|                    2|
|2004|                    2|
|2003|                    2|
|2002|           